<a href="https://colab.research.google.com/github/Jacobanderson42/Project1/blob/master/Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Calculate ssp and plot each dive

In [0]:
def CTDdives(f):
  import glob
  import pandas as pd
  import io
  import matplotlib.pyplot as plt
  import numpy as np
  import requests
  import datetime as dt
  import matplotlib.dates as md
  from datetime import date
  import math
  import statistics
  from google.colab import files
  url = 'https://raw.githubusercontent.com/Jacobanderson42/Project1/master/Data_Files/'+f
  b = f[:-4]
  data = pd.read_csv(url)
  temp = data['seawater_temperature']
  sal = data['practical_salinity']
  depth = data['seawater_pressure']
  time = data['time']
  cutoff = .95
  res = 50
  res_list = [] 
  c = 1449.2+4.6*temp-0.055*temp*temp+0.00029*temp*temp*temp+(1.34-0.01*temp)*(sal-35)+0.016*depth
  for i in range(round(len(depth)/res)-1) :      
    if depth[res*i] > max(depth)*cutoff and depth[res*(i+1)] < max(depth)*cutoff:
      res_list.append(res*i)
    elif depth[res*i] < max(depth)*cutoff and depth[res*(i+1)] > max(depth)*cutoff and res_list != []:
      res_list.append(res*i)
  if (len(res_list) % 2) == 1 :
      res_list.pop()
  dives = []
  dives_ssc = []
  fig = plt.figure()
  plt.rcParams['xtick.bottom'] = plt.rcParams['xtick.labelbottom'] = False
  plt.rcParams['xtick.top'] = plt.rcParams['xtick.labeltop'] = True
  for i in range(int((len(res_list))/2)):
    x = [depth[res_list[2*i]:res_list[2*i+1]]]
    y = [c[res_list[2*i]:res_list[2*i+1]]]
    plt.title(b+' Dives')
    plt.scatter(y,x, s=.5, label=i+1)
    plt.ylabel("depth (m)")
    plt.xlabel("ssp (m/s)")
    plt.legend(loc="lower right", title="Dive")
  plt.gca().invert_yaxis()
  plt.show()
  return fig

Plot average ssp

In [0]:
def CTDave(f):
  import glob
  import pandas as pd
  import io
  import matplotlib.pyplot as plt
  import numpy as np
  import requests
  import datetime as dt
  import matplotlib.dates as md
  from datetime import date
  import math
  import statistics
  from google.colab import files
  url = 'https://raw.githubusercontent.com/Jacobanderson42/Project1/master/Data_Files/'+f
  b = f[:-4]
  data = pd.read_csv(url)
  temp = data['seawater_temperature']
  sal = data['practical_salinity']
  depth = data['seawater_pressure']
  time = data['time']
  c = 1449.2+4.6*temp-0.055*temp*temp+0.00029*temp*temp*temp+(1.34-0.01*temp)*(sal-35)+0.016*depth
  cutoff = .95
  res = 50
  res_list = [] 
  for i in range(round(len(depth)/res)-1) :     
      if depth[res*i] > max(depth)*cutoff and depth[res*(i+1)] < max(depth)*cutoff:
        res_list.append(res*i)
      elif depth[res*i] < max(depth)*cutoff and depth[res*(i+1)] > max(depth)*cutoff and res_list != []:
        res_list.append(res*i)
  if (len(res_list) % 2) == 1 :
      res_list.pop()
  dives = []
  dives_ssc = []
  for i in range(int((len(res_list))/2)):
    x = [depth[res_list[2*i]:res_list[2*i+1]]]
    y = [c[res_list[2*i]:res_list[2*i+1]]]
    dives.append(x)
    dives_ssc.append(y)
  stp = 1
  minim = math.floor(min([min(dives[i][0]) for i in range(len(dives))]))
  maxim = math.ceil(max([max(dives[i][0]) for i in range(len(dives))]))
  depths = np.arange(minim, maxim, stp).tolist()
  depcalc = []
  sspcalc = []
  x = []
  y = []
  for j in range(len(depths)-1):  
    for i in range(len(dives)):
      d = np.array(dives[i])
      ssp = np.array(dives_ssc[i])
      ind = np.where(np.logical_and(d>=depths[j], d<=depths[j+1]))
      dep = d[ind]
      ssp = ssp[ind]
      e = np.mean(dep)
      r = np.mean(ssp)
      depcalc.append(e)
      sspcalc.append(r)
    cleaneddep = [x for x in depcalc if (math.isnan(x) == False)]
    cleanedssp = [x for x in sspcalc if (math.isnan(x) == False)]
    avdepth = np.mean(cleaneddep)
    avssp = np.mean(cleanedssp)
    x.append(avdepth)
    y.append(avssp)
    depcalc = []
    sspcalc = []
  plt.rcParams['xtick.bottom'] = plt.rcParams['xtick.labelbottom'] = False
  plt.rcParams['xtick.top'] = plt.rcParams['xtick.labeltop'] = True
  fig = plt.figure()
  plt.title(b+' Average')
  plt.scatter(y,x,s=.6)
  plt.ylabel("depth (m)")
  plt.xlabel("ssp (m/s)")
  plt.gca().invert_yaxis()
  plt.show()
  return fig

Plot ssp for day vs night

In [0]:
def CTDdaynight(f):
  import glob
  import pandas as pd
  import io
  import matplotlib.pyplot as plt
  import numpy as np
  import requests
  import datetime as dt
  import matplotlib.dates as md
  from datetime import date
  import math
  import statistics
  from google.colab import files
  url = 'https://raw.githubusercontent.com/Jacobanderson42/Project1/master/Data_Files/'+f
  b = f[:-4]
  data = pd.read_csv(url)
  temp = data['seawater_temperature']
  sal = data['practical_salinity']
  depth = data['seawater_pressure']
  time = data['time']
  c = 1449.2+4.6*temp-0.055*temp*temp+0.00029*temp*temp*temp+(1.34-0.01*temp)*(sal-35)+0.016*depth
  timestamps = time-2.208e+9
  d=[dt.datetime.fromtimestamp(ts) for ts in timestamps]
  dhour = [i.hour for i in d]
  dmin = [i.minute for i in d]
  date = dt.date.fromtimestamp(timestamps[0])
  year = date.year
  month = date.month
  day = date.day
  date = '{year}-{month}-{day}'.format(year=year,month=month,day=day)
  r = requests.get('http://api.sunrise-sunset.org/json?lat=46.00000&lng=-145.00000&date={}&formatted=0'.format(date))
  dat = r.json()
  sunrisehour = int(dat['results']['sunrise'][11:13])
  sunrisemin = int(dat['results']['sunrise'][14:16])
  sunsethour = int(dat['results']['sunset'][11:13])
  sunsetmin = int(dat['results']['sunset'][14:16])
  daydepth = []
  dayssp = []
  nightdepth = []
  nightssp = []
  for i in range(len(dhour)):
    if (dhour[i]>sunrisehour and dmin[i]>sunrisemin) or (dhour[i]<sunsethour and dmin[i]<sunsetmin):
      daydepth.append(depth[i])
      dayssp.append(c[i])
    else:
      nightdepth.append(depth[i])
      nightssp.append(c[i])
  plt.rcParams['xtick.bottom'] = plt.rcParams['xtick.labelbottom'] = False
  plt.rcParams['xtick.top'] = plt.rcParams['xtick.labeltop'] = True
  fig = plt.figure()
  plt.title(b+' Day vs Night')
  plt.scatter(dayssp,daydepth,s=.6,label='day')
  plt.scatter(nightssp,nightdepth,s=.6,label='night')
  plt.ylabel("depth (m)")
  plt.xlabel("ssp (m/s)")
  plt.legend(loc="lower right", title="Dive")
  plt.gca().invert_yaxis()
  plt.show()
  return fig

Call functions

In [0]:
f = ['AxialDeep_Summer.csv','AxialDeep_Winter.csv','AxialShallow_Summer.csv','AxialShallow_Winter.csv','OregonOffshoreDeep_Summer.csv','OregonOffshoreDeep_Winter.csv',
     'OregonOffshoreShallow_Summer.csv','OregonOffshoreShallow_Winter.csv','OregonShelf_Summer.csv','OregonShelf_Winter.csv','OregonSlopeDeep_Summer.csv',
     'OregonSlopeDeep_Winter.csv','OregonSlopeShallow_Summer.csv','OregonSlopeShallow_Winter.csv']
for i in f:
  CTDdives(i) 
  CTDave(i)
  CTDdaynight(i)